In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import midi_manipulation
from keras.models import Model
from keras.layers import Input, LSTM, Dense

from utils import *

tf.reset_default_graph()
tf_checks()


# Get Input Song Matrixes and Target Song Matrixes
input_songs, target_songs = get_song_matrixes('./blues', 176, 500)


# Finding the longest song in the dataset
# Finding the number of tokens in the songs (usually 156)
max_encoder_seq_length, num_encoder_tokens, \
    max_decoder_seq_length, num_decoder_tokens = get_data_insights(input_songs, target_songs)


print('Number of samples:', len(input_songs))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


# Get input data in shape (num_sample, max_seq_length, num_tokens)
encoder_input_data, decoder_input_data, decoder_target_data = get_input_data(
                                                                        input_songs, 
                                                                        target_songs,
                                                                        max_encoder_seq_length, 
                                                                        num_encoder_tokens, 
                                                                        max_decoder_seq_length, 
                                                                        num_decoder_tokens)

Using TensorFlow backend.



TensorFlow Version: 1.1.0

Default GPU Device: /gpu:0



[*] Converting songs to matrix


 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋       | 175/180 [00:31<00:00,  5.62it/s]

[*] Converted 176 songs to matrix



Number of samples: 141
Number of unique input tokens: 156
Number of unique output tokens: 156
Max sequence length for inputs: 500
Max sequence length for outputs: 500

Encoder input data shape: (141, 500, 156)
Decoder input data shape: (141, 500, 156)
Decoder target data shape: (141, 500, 156)



In [2]:
# Hyperparameters
batch_size  =   32      # Batch size for training.
epochs      =   5     # Number of epochs to train for.
latent_dim  =   156     # Latent dimensionality of the encoding space.
#num_samples =   10000   # Number of samples to train on.


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs)



# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print()
print(model.summary())
print()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 156)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 156)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 156), (None, 195312      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 156),  195312      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
         

In [3]:
# Run training
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

print('[*] Starting Training')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
# model.load_weights('s2s.h5')
print('[*] Ready to be used \n\n')
# Save model
model.save('s2s.h5')



# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

[*] Starting Training
Epoch 1/5
141/141 [==============================] - ETA: 9s - loss: 2.5033e-04 - acc: 1.000 - ETA: 4s - loss: 0.0072 - acc: 0.9994    - ETA: 2s - loss: 0.0078 - acc: 0.999 - ETA: 0s - loss: 0.0060 - acc: 0.999 - 8s 54ms/step - loss: 0.0103 - acc: 0.9987
Epoch 2/5
128/141 [==========================>...] - ETA: 4s - loss: 0.0210 - acc: 0.996 - ETA: 2s - loss: 0.0132 - acc: 0.997 - ETA: 1s - loss: 0.0089 - acc: 0.998 - ETA: 0s - loss: 0.0084 - acc: 0.9985

 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋       | 175/180 [00:50<00:01,  3.50it/s]

141/141 [==============================] - 6s 42ms/step - loss: 0.0076 - acc: 0.9987
Epoch 3/5
141/141 [==============================] - ETA: 4s - loss: 0.0064 - acc: 0.998 - ETA: 2s - loss: 0.0054 - acc: 0.999 - ETA: 1s - loss: 0.0057 - acc: 0.998 - ETA: 0s - loss: 0.0073 - acc: 0.998 - 6s 41ms/step - loss: 0.0066 - acc: 0.9987
Epoch 4/5
141/141 [==============================] - ETA: 4s - loss: 0.0098 - acc: 0.998 - ETA: 2s - loss: 0.0079 - acc: 0.998 - ETA: 1s - loss: 0.0073 - acc: 0.998 - ETA: 0s - loss: 0.0069 - acc: 0.998 - 6s 43ms/step - loss: 0.0063 - acc: 0.9987
Epoch 5/5
141/141 [==============================] - ETA: 4s - loss: 2.0674e-04 - acc: 1.000 - ETA: 2s - loss: 0.0031 - acc: 0.9994    - ETA: 1s - loss: 0.0021 - acc: 0.999 - ETA: 0s - loss: 0.0030 - acc: 0.999 - 6s 42ms/step - loss: 0.0061 - acc: 0.9987
[*] Ready to be used 




C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\topology.py:2344: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 156) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 156) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [21]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    song_matrix = np.zeros(
                        (max_decoder_seq_length, 
                        num_decoder_tokens),
                        dtype='float32')
    i = 0
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        idx = np.argmax(output_tokens[-1,-1,:])
        #print(output_tokens[-1,-1,:])
        song_matrix[i, idx] = 1
        target_seq[0, 0, idx] = 1.
        #print(np.array(h).shape)
        # Exit condition: either hit max length
        # or find stop character.
        if (i+2 > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        
        

        if i%250 == 0:
            print('[iter:{}] [max_decoder_seq_length: {}]'.format(i, max_decoder_seq_length))
        i+=1

        # Update states
        states_value = [h, c]

    return song_matrix

In [22]:
# Take one sequence (part of the training test)
# for trying out decoding.
print('[*] Encoding-Decoding')
input_seq = encoder_input_data[0:-1]
decoded_song = decode_sequence(input_seq)
print('-')
print('Input sentence:', input_songs[0][40])
print('Decoded sentence:', decoded_song[40])

print(input_songs[0]==decoded_song)



# Converting Song to midi from matrix
midi_manipulation.noteStateMatrixToMidi(decoded_song)

[*] Encoding-Decoding
[iter:0] [max_decoder_seq_length: 500]
[iter:250] [max_decoder_seq_length: 500]
-
Input sentence: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0
 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
Decoded sentence: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. 

In [17]:
midi_manipulation.noteStateMatrixToMidi(decoder_input_data[0])

In [23]:
from sklearn.metrics import accuracy_score

accuracy_score(encoder_input_data[0], decoded_song)

0.002